

Environment: Python 3.7.3 and Jupyter notebook

Libraries used: 

* pandas (for dataframe, included in Anaconda Python 3.7) 
* re (for regular expression, included in Anaconda Python 3.7) 

## Introduction

This project involved extraction of specified data from a semi-structured xml file containing US patent data records and to export it into a specified format as a csv and a json file. 

Following steps were taken in order to complete this project. 

1. Analysis of sample input and sample output files to identify the relationship between the structure, content and format of input file and output files. 

2. Identification of consistent patterns to assist in use of regular expressions to select and extract required data.

3. Transformation of extracted data into required format and writing it to files. 


### Step 1

#### Analysis of sample input and output files

Following strategies were employed in searching for the location of required data in the sample input file:
* searching for field title or part of. 
* searching for field content or part of. 
* number of occurances of same or similar information in the record
* occurances of false matches

In [1]:
# reading sample output
sample_output = open('sample_output.csv','r').readlines()

#first line contains headers of all the columns that need to be included.
print(sample_output[0])

grant_id,patent_title,kind,number_of_claims,inventors,citations_applicant_count,citations_examiner_count,claims_text,abstract



## 1.  Import libraries 

In [2]:
# Code to import libraries 

import re
import pandas as pd

## 2. Parse XML File

Entire text file is opened and read into raw_data. 

The information that we need from each record is within tags `<us-patent-grant...>` and `</us-patent-grant>`. 
All records are extracted from raw_data using the findall function and stored as a list of strings in records. 
(.*?) pattern captures all content between the tags including the newline character. 

In [3]:
# reading the input file
raw_data = open('Group163.txt','r').read()

# creating a list, containing all records individually separated
records = re.findall(r'<us-patent-grant(.*?)/us-patent-grant', str(raw_data),re.S)


In [1]:
pip install pandoc

  Running setup.py install for pandoc: started
    Running setup.py install for pandoc: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### General Method

For each field mentioned previously, a for loop is used to search each record in the list of records and the required information is selected for extraction using an appropriate regular expression. Extracted information is then stored in a list specific for that field. 

More specific method is discussed for each field.

Each regular expression was tested for accuracy and specificity of text extracted using the sample input and output files. 

#### grant_id

Each record is identified using a unique grant id. The location of grant id in the input file was found by searching for specific grant ids from the sample output. 

Regular expression used for this search is '"(\w*)-2'. This expression uniquely captures the grant id from the file name. 

In [4]:
# list to store grant id for each record.
grant_id = []

# for loop to iterate over all records to extract grant ids and append to the list.
for record in records:
    grant_id.append(
        re.search(r'"(\w*)-2',record)
        .group(1))

#### patent_kind

Following approach was used to extract the patent kind information.

Comparison of sample input and output showed that there is one to one relationship between a two digit code consisting of a letter and number between tags `<kind>` and `</kind>` in the input file and descriptive text in the sample output file. A dictionary linking these two digit codes with respective patent kind description was created. 

The patent kind code for each record was extracted using the regular expression: `>(\w+)<\/kind>(?!<name>)`
This was then replaced with the patent kind description using the dictionary.

In [5]:
# list to store patent kind of each record.
patent_kind = []

# dictionary linking the two digit patent kind code and the descriptive text required for output.
kind_dict = {"B1":"Utility Patent Grant (no published application) issued on or after January 2, 2001.", "B2":"Utility Patent Grant (with a published application) issued on or after January 2, 2001.","S1":"Design Patent","E1":"Reissue Patent","P2":"Plant Patent Grant (no published application) issued on or after January 2, 2001","P3":"Plant Patent Grant (with a published application) issued on or after January 2, 2001"}

# extracting code for patent kind for each record and replacing it with the descriptive text using the dictionay.
for record in records:
    kind = re.search(r'>(\w+)</kind>(?!<name>)', record).group(1)
    for key in kind_dict.keys():
        kind = kind.replace(key,kind_dict[key])
    patent_kind.append(kind)

#### patent_title

Patent title was conveniently stored between tags `<invention-title....>` and `</invention-title>`. 

Regular expression '>(.*?)</invention-title>' was used for extracting patent title from each record.

In [6]:
# list to store patent title for each record.
patent_title = []

# for loop to extract patent title from each record and append to the list.
for record in records:
    patent_title.append(
        re.search(r'>(.*?)</invention-title>', record)
        .group(1))

#### no_of_claims

No of claims was also conveniently stored between tags `<number-of-claims>` and `</number-of-claims>`. 

Regular expression `<number-of-claims>(.*?)</number-of-claims>` was used for extracting no of claims from each record.

In [7]:
# list to store no of claims for each record.
no_of_claims = []

# for loop to extract no of claims from each record and append to the list.
for record in records:
    no_of_claims.append(re.search(r'<number-of-claims>(.*?)</number-of-claims>', record).group(1))

#### inventor_name

A record can contain one to many inventor names. Inventor names are also stored separately as first and last names within tags `<first-name>...</first-name>` and `<last-name>...</last-name>` respectively for each inventor, whose information is stored within tags `<inventor...>...</inventor>`.

Last names for all inventors for a record were extracted as a list of strings using the findall function and the regular expression `<inventor.*?<last-name>(.*?)</last-name>`. First names were also extracted in a similar way. 

Two list of lists containing last names and first names of inventors for each record were created. 

The first and last name of each inventor was concatenated by iterating over each name within list of names within list for all records. 

In order to transform this into the format given in sample output, inventor names for each record were stored in a string with square brackets containing all inventor names for that record separated by a comma. .join() function and string formatting were used to achieve this.  


In [8]:
# list to store lists that contain first names of all inventors for each record.
inventor_first_name = []

# list to store lists that contain last names of all inventors for each record.
inventor_last_name = []

# list to store lists that contain full names of all inventors for each record obtained by concatenating
# first and last names from the lists above. 
full_name = []

# list to store lists that contains full names of all inventors in the format required for output.
inventor_name = []

# for loop to extract list of first names and last names from each record and append to the respective list.
for record in records:
    inventor_last_name.append(re.findall(r'<inventor.*?<last-name>(.*?)</last-name>', record, re.S))

    inventor_first_name.append(re.findall(r'<inventor.*?<first-name>(.*?)</first-name>', record, re.S))

# concatenating first and last names
for i in range(len(inventor_first_name)):
    full_name.append([])
    for j in range(len(inventor_first_name[i])):
        full_name[i].append((inventor_first_name[i][j]) + ' ' + (inventor_last_name[i][j]))

# transforming list contents into the format required for output
for name in full_name:
    inventor_name.append("[%s]" % (','.join(name)))

#### citations_applicant_count & citations_examiner_count

It is specified for each citation in a record whether the citation is by the applicant or examiner. 
In order to obtain the number of citations by applicant, all instances of 'cited by applicant' are extracted in a list for each record and stored as the length of that list within the citations_applicant_count list. Similar method is used for citations by examiners. 

In [9]:
# list to store the number of citations cited by the applicant
citations_applicant_count = []

# Using a for loop to extract all occurances of 'cited by applicant' for a record and storing the length of that list as count.
for record in records:
    citations_applicant_count.append(len(re.findall(r'cited by applicant', record)))
    
# list to store the number of citations cited by the examiner
citations_examiner_count = []

# Using a for loop to extract all occurances of 'cited by examiner' for a record and storing the length of that list as count.
for record in records:
    citations_examiner_count.append(len(re.findall(r'cited by examiner', record)))
    
    

#### abstract

Abstract for each reacord is stored within tags `<abstract...>...</abstract>`. Within these tags multiple sub tags are used for identification of different parts of abstract, for example each paragraph of abstract is stored within `<p id...>...</p>`. A regular expression called internal tag is used to remove these sub tags but keep the abstract content within these tags. Another regular expression called new_line_tag is defined to remove all new line characters from within the abstract. 
These tags are removed by replacing them with an empty string using .sub() function. 
All abstract text is extracted from each record using the regular expression `<abstract.*?>(.*?)</abstract>`. When a record does not contain any abstract i.e searching of `<abstract.*?>(.*?)</abstract>` within a record yields `None`, 'NA' is appended to the list of abstracts. For the records containing abstract text, the text is cleaned up as described previously. This is then appended to the list of abstracts.


In [10]:
# list to store all abstract text for each record. 
abstract = []

# Regular expression defined to remove all internal tags from within the abstract text.
internal_tag = re.compile(r'<[^>]+>')

# Regular expression defined to remove all new line characters from within the abstract text.
new_line_tag = re.compile(r'[\n]')

# for loop to extract all abstract text for each record, append 'NA' if no abstract is given for a record and
# append cleaned abstract text to list where abstract is given. 

for record in records:
    raw_abstract = re.search(r'<abstract.*?>(.*?)</abstract>', record, re.S)
    if raw_abstract == None :
        abstract.append('NA')
    else :
        internal_tag_removed = internal_tag.sub('',raw_abstract.group(1))
        new_line_tag_removed = new_line_tag.sub('',internal_tag_removed)
        abstract.append(new_line_tag_removed)


#### claim_text

Each record has multiple claims where each claim is recorded between tags `<claim id...>...</claim>`. Each claim has its own claim text, this however may be present as separate pieces of text, where each bit of claim text is enveloped between the tags `<claim-text>...</claim-text>` or `<claim-text>...\n`.

After extraction of claim text for all claims within a record using the regular expression `<claim id.*?>(.*?)</claim>`, we obtain a list of strings, where each string contains claim text for one of the claims within the record. From within each of these strings, claim text excluding all other information is extracted as list of strings using regular expression `<claim-text>(.*?)(?:</claim-text>)?[\n]`. 

These list of strings are then joined together to form a string containing claim text for a claim. All strings of claim texts for all claims within a record are then joined together. The joins are performed using the .join() function. 

All internal tags are removed with a method similar to the one used for abstract. 

In order to transform this into the format given in sample output, claim text for each claim within each record were stored in a string with square brackets containing all claim text for all claims for that record separated by a comma. .join() function and string formatting were used to achieve this.  



In [11]:
# list to store claim text for all records. 
claim_text = []

#for loop to extract claim text for each record and transform it into the format required for output
for record in records:
    
    claim_text_raw = []
    claim_text_concat = []
    claim_text_records = []
    
    # extracting text for all claims within a record
    claims_block = re.findall(r'<claim id.*?>(.*?)</claim>', record,re.S)
    
    # extracting only claim text from within each claim 
    for claim in claims_block:
        claim_text_raw.append(re.findall(r"<claim-text>(.*?)(?:</claim-text>)?[\n]", claim,re.S))
    
    # Joining together all text for a claim
    i = 0
    while i<len(claim_text_raw):
        claim_text_concat.append(''.join(str(x) for x in claim_text_raw[i]))
        i+=1
        
    # joining together all claims for a record, separated by a comma.    
    claim_text_records.append(','.join(str(x) for x in claim_text_concat))
    
    # replacing internal tags with empty string
    internal_tag_removed = internal_tag.sub('', claim_text_records[0])
    
    # replacing new line character with empty string
    new_line_tag_removed = new_line_tag.sub('',internal_tag_removed)
    
    # transforming all claim text for a record to the required format and appending it to the list.
    claim_text.append("[%s]"%(new_line_tag_removed))

    


### Data extraction and transformation complete

At this stage, all required data is extracted into a field specific list in the required format. 

In [12]:
print("Grant ID:", grant_id[:4], '\n\n\n', "Patent Title:", patent_title[:4], '\n\n\n', "Kind:", patent_kind[:4], 
      '\n\n\n', "No of claims:", no_of_claims[:4] , '\n\n\n', "Inventor:", inventor_name[:4], 
      '\n\n\n', "Citations Applicant Count:", citations_applicant_count[:4], 
      '\n\n\n', "Citations Examiner Count:", citations_examiner_count[:4], '\n\n\n', "Claim Text:", claim_text[:4],
     '\n\n\n', "Abstract:", abstract[:4])


Grant ID: ['USD0854286', 'US10357242', 'US10361198', 'US10359952'] 


 Patent Title: ['Shoe', 'Suture delivery device', 'Integrated circuit devices and method of manufacturing the same', 'Method and system for implementing writable snapshots in a virtualized storage environment'] 


 Kind: ['Design Patent', 'Utility Patent Grant (with a published application) issued on or after January 2, 2001.', 'Utility Patent Grant (with a published application) issued on or after January 2, 2001.', 'Utility Patent Grant (no published application) issued on or after January 2, 2001.'] 


 No of claims: ['1', '8', '8', '21'] 


 Inventor: ['[John Hlavacs]', '[Michi E. Garrison,Gregory M. Hyde,Richard J. Renati,Alan K. Schaer,Tony M. Chou]', '[Sidharth Rastogi,Subhash Kuchanuri,Raheel Azmat,Pan-jae Park,Chul-hong Park,Jae-seok Yang,Kwan-young Chun]', '[Mohit Aron,Brian Byrne]'] 


 Citations Applicant Count: [10, 320, 11, 258] 


 Citations Examiner Count: [34, 0, 0, 4] 


 Claim Text: ['[The ornament

## 3. Export CSV file

Pandas is used to create a dataframe using all lists created from the previous section. 

Pandas has a convenient method .to_csv() to write a dataframe to a csv file. 

In [13]:
# Creating a dataframe using pandas
df = pd.DataFrame(data={'grant_id': grant_id,'patent_title': patent_title,'kind': patent_kind,'number_of_claims': no_of_claims,'inventors': inventor_name,'citations_applicant_count': citations_applicant_count,'citations_examiner_count': citations_examiner_count,'claims_text': claim_text,'abstract': abstract})
print(df[0:4])

# creating a csv file from the dataframe
df.to_csv("Group163final.csv", sep=',',index=False)

     grant_id                                       patent_title  \
0  USD0854286                                               Shoe   
1  US10357242                             Suture delivery device   
2  US10361198  Integrated circuit devices and method of manuf...   
3  US10359952  Method and system for implementing writable sn...   

                                                kind number_of_claims  \
0                                      Design Patent                1   
1  Utility Patent Grant (with a published applica...                8   
2  Utility Patent Grant (with a published applica...                8   
3  Utility Patent Grant (no published application...               21   

                                           inventors  \
0                                     [John Hlavacs]   
1  [Michi E. Garrison,Gregory M. Hyde,Richard J. ...   
2  [Sidharth Rastogi,Subhash Kuchanuri,Raheel Azm...   
3                           [Mohit Aron,Brian Byrne]   

   citations

## 4. Export JSON file 

In order to complete this task, the sample output file in JSON version was checked for the JSON structure. JSON structure works as key-value pairs. In the output file, grant_id for each record was used as its key. The set of remaining fields for the record in a key-value structure with field title as the key and the record data as its value was recorded as the value for each grant_id. 

All field titles are stored in a list. 

1. All the key-value pairs of fields titles and its content (excluding Grant ID) for the record is stored within curly brackets as a string for each record within a list. 

2. All grant id and the respective string from step 1 are then stored as key value pairs as a string for each record in a list. 

3. All elements in the list list from step 2 are joined into one single string with each element separated by a comma using the .join() function and the entire string placed into a set of curly brackets. 

4. The string from step 3 is then written into a .json file. 

In [1]:
# list of all field titles
headers = ["patent_title","kind","number_of_claims","inventors","citations_applicant_count","citations_examiner_count","claims_text","abstract"]

# list to store all key value pairs (except grant id) for all records
key_value_level2 = []

i = 0
while i < len(grant_id):
    key_value_level2.append('{"'
        +str(headers[0])+'":"'+str(patent_title[i])+'","'
        +str(headers[1])+'":"'+str(patent_kind[i])+'","'
        +str(headers[2])+'":'+str(no_of_claims[i])+',"'
        +str(headers[3])+'":"'+str(inventor_name[i])+'","'
        +str(headers[4])+'":'+str(citations_applicant_count[i])+',"'
        +str(headers[5])+'":'+str(citations_examiner_count[i])+',"'
        +str(headers[6])+'":"'+str(claim_text[i])+'","'
        +str(headers[7])+'":"'+str(abstract[i])+'"}')
    i+=1

# list to store all key value pairs of grant-id and its respective content from the list key_value_level2
key_value_level1 = []

j = 0  
while j < len(grant_id):
    key_value_level1.append('"'+str(grant_id[j])+'":'+str(key_value_level2[j]))
    j+=1

# Joining key value pairs of all records separated by a comma into a single string within curly brackets
json_string = "{"+",".join(key_value_level1)+"}"

#writing the json format string into a file
f= open("Group163final.json","w+")
f.write(str(json_string))
f.close()

NameError: name 'grant_id' is not defined

## 5. Summary


#### hex code

Certain characters in the input file are encoded in the HTML hex code. It appears from reviewing the sample input and output files that the code written to create those files is able to handle the hex code and convert it to the respective character when viewed in a text editor. We managed to figure out a way to do this as described below. We have described here the approach we used to deal with the codes, however given that we were not required to handle these codes and at this stage had not solved all remaining codes, we decided to leave them as is. 

The solution we developed was to compile a regular expression for the hex code in the input file and then replace it with the python encoding for the unicode character. 

example: 
hex code - &#x2018
python unicode - u"\u2018"

code_re = re.compile(r'&#x2018')
transformed_string = code_re.sub(u"\u2018",str'string to be converted')


## 6. References

- Biek M. (2008, September 4). *How would you make a comma-separated string from a list?* [Response to]. Retrieved from http://stackoverflow.com/a/44781

- pandas: powerful Python data analysis toolkit. (2019). *pandas 0.25.1 documentation: pandas.DataFrame.to_csv*. Retrieved from https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html

- Python 3.7.4 documentation. (2019). *re — Regular expression operations* . Retrieved from https://docs.python.org/3/library/re.html#module-re
